In [1]:
import pandas as pd
import glob
import os
import re
import psycopg2 as pg
import fiona

# Path to the jgeo root
jgeo_path = '/home/greg/data/rawdata/JornadaGeospatial/'

## Populate the jrn_studies schema from latest_all_jornada_studies_gdb

This notebook takes the layer names from the project geopackages and copies the layers from
latest_all_jornada_studies schema into the jrn_studies schema. In the process it gives them
consistent layer naming. This would come after populating jgeo.latest_all_jornada_studies
using the "upload_gdb_to_jgeo" notebook.

In [2]:
# Get a list of all project geopackages
gpkg_path = os.path.join(jgeo_path, 'jrn_studies_gpkg')
gpkgs = glob.glob(os.path.join(gpkg_path, 'prj*.gpkg'))
#gpkgs

In [3]:
print(gpkgs[1])
all_l = fiona.listlayers(gpkgs[1])

/home/greg/data/rawdata/JornadaGeospatial/jrn_studies_gpkg/prj187.gpkg


In [4]:
all_l

['JornadaStudy_187_experimental_plot']

In [5]:
# Database cred and connection
import sys
sys.path.append('/home/greg/admin/')
import jgeo_cred
connection_psql = pg.connect(user=jgeo_cred.user, password=jgeo_cred.pwd,
                             port="5432", database=jgeo_cred.db, host=jgeo_cred.host)

In [112]:
# Replace these phrases with prjXXX
rexpression = '^JornadaStudy_|^JonradaStudy_|^Study|^JornadaResearch_|^JornadaReasearch_|^JornadaReseach_'
for g in gpkgs:
    print(g)
    # Get filename and split out the project id
    prj = os.path.basename(g).split('.')[0]
    layers_raw = fiona.listlayers(g)
    # Replace the expressions above with "prj"
    layers_ed = [re.sub(rexpression, 'prj', lay).lower() for lay in layers_raw]
    # If prj is still missing from layer, add the prefix
    layers_pad = [prj + '_' + lay if prj not in lay else lay for lay in layers_ed]
    print(layers_raw)
    print(layers_pad)
    
    # Now loop through and create a table in the new schema and copy the data in
    cursor_psql = connection_psql.cursor()
    for lnum, layer in enumerate(layers_pad):
        create_query = 'create table jrn_studies.{0} (like all_studies_jornada_gdb.{1} including all);'.format(layer, layers_raw[lnum])
        insert_query = 'insert into jrn_studies.{0} select * from all_studies_jornada_gdb.{1};'.format(layer, layers_raw[lnum])
        cursor_psql.execute(create_query)
        cursor_psql.execute(insert_query)
    
    connection_psql.commit()
    cursor_psql.close()

./Jornada_prj_gpkg/prj468.gpkg
['JornadaStudy_468_PlotPitfallPts']
['prj468_plotpitfallpts']
./Jornada_prj_gpkg/prj187.gpkg
['JornadaStudy_187_experimental_plot']
['prj187_experimental_plot']
./Jornada_prj_gpkg/prj011.gpkg
['JornadaStudy_011_npp_quad', 'JornadaStudy_011_npp_sites']
['prj011_npp_quad', 'prj011_npp_sites']
./Jornada_prj_gpkg/prj486.gpkg
['JornadaStudy_486_Site_locations']
['prj486_site_locations']
./Jornada_prj_gpkg/prj291.gpkg
['JornadaStudy_291_OuterBnd', 'JornadaStudy_291_OuterPlotBnd', 'JornadaStudy_291_GrassEstablishmtLimits']
['prj291_outerbnd', 'prj291_outerplotbnd', 'prj291_grassestablishmtlimits']
./Jornada_prj_gpkg/prj188.gpkg
['JornadaStudy_188_Collection_Site', 'JornadaStudy_188_CollectionArea']
['prj188_collection_site', 'prj188_collectionarea']
./Jornada_prj_gpkg/prj488.gpkg
['JornadaStudy_488_proposed_sites', 'JornadaStudy_488_study_sites']
['prj488_proposed_sites', 'prj488_study_sites']
./Jornada_prj_gpkg/prj168.gpkg
['JornadaStudy_168_pts']
['prj168_pts'